In [1]:
import warp as wp
import numpy as np
import trimesh
import polyscope as ps
import gpytoolbox as gpy

from warp.sim.collide import TriMeshCollisionDetector

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.


In [2]:
device = wp.get_device('cuda')
# if you don't have a CUDA-compatible GPU try switching to CPU
# device = wp.get_device('cpu')

Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.6
   Devices:
     "cpu"      : "Intel64 Family 6 Model 186 Stepping 2, GenuineIntel"
     "cuda:0"   : "NVIDIA GeForce RTX 3050 6GB Laptop GPU" (6 GiB, sm_86, mempool enabled)
   Kernel cache:
     C:\Users\joana\AppData\Local\NVIDIA\warp\Cache\1.8.0


In [3]:
mesh = trimesh.load("C:/Users/joana/Downloads/Task_1_Bounds_computation/Triangular_Meshes/A00000500.ply")

In [4]:
def viz_mesh():
    V, F = gpy.read_mesh('C:/Users/joana/Downloads/Task_1_Bounds_computation/Triangular_Meshes/A00000500.ply')
    ps.init()
    ps.register_surface_mesh('cube', V, F)
    # pc = ps.register_point_cloud('vertices', V)
    # pc.add_scalar_quantity("vertex_index", np.arange(V.shape[0]), enabled=True, datatype='categorical')
    ps.screenshot("trimesh.png")
    ps.show()
    return V, F
V, F = viz_mesh()


In [5]:
builder = wp.sim.ModelBuilder()

# converts vertex data from an array to warp's vec3 type
vertices = [wp.vec3(mesh.vertices[i,:]) for i in range(mesh.vertices.shape[0])]

# sets up cloth simulation parameters
builder.add_cloth_mesh(
            pos=wp.vec3(0.0, 0.0, 0.0), # position of the mesh
            rot=wp.quat_identity(), # rotation of the mesh
            scale=1.0, # size
            vertices=vertices,
            indices=mesh.faces.reshape(-1),
            vel=wp.vec3(0.0, 0.0, 0.0),
            density=0.02, #mass per volume
            tri_ke=1.0e5, # stiffness of the triangles
            tri_ka=1.0e5,
            tri_kd=2.0e-6,
            edge_ke=10,
)
builder.color()
model = builder.finalize()

Warp UserWarning: The graph is not optimizable anymore, terminated with a max/min ratio: 2.0 without reaching the target ratio: 1.1
Module warp.sim.graph_coloring 3c9ae71 load on device 'cpu' took 65.45 ms  (cached)


In [6]:
# to access ForceElementAdjacencyInfo, you need to construct a VBDIntegrator (you dont need to understand what it is)
# variational block descent
vbd_integrator = wp.sim.VBDIntegrator(model)

Module warp.sim.integrator_vbd 715ddb2 load on device 'cpu' took 23.32 ms  (cached)


In [7]:
colision_detector = TriMeshCollisionDetector(model)

Module warp.sim.collide 3b78b77 load on device 'cuda:0' took 32.15 ms  (cached)


In [8]:
# configuring the collision detection system with specific distance thresholds
#vertex detection within 5 units of triangle surfaces
colision_detector.vertex_triangle_collision_detection(5.0)

# detects when cloth edges come within 5 units
colision_detector.edge_edge_collision_detection(5.0)

In [9]:
# d^v_{min}
print(colision_detector.vertex_colliding_triangles_min_dist)
# d^E_{min}
print(colision_detector.edge_colliding_edges_min_dist)
# d^T_{min}
print(colision_detector.triangle_colliding_vertices_min_dist)

[2.0408168 1.3784707 1.7306733 ... 1.7102327 1.3814771 2.0408168]
[1.3784707 1.8084555 1.514926  ... 1.3814771 2.012112  1.515333 ]
[1.4946659 1.3784707 1.6526986 ... 1.6386598 1.3814771 1.5250734]


In [10]:
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo
# how to iterate over neighbor elements
@wp.kernel
def iterate_vertex_neighbor_primitives(
    adjacency: ForceElementAdjacencyInfo
):
    particle_idx = wp.tid()

    # iterating over neighbor faces
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, particle_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, particle_idx, face_counter)
    # iterating over neighbor edges
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, particle_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, particle_idx, edge_counter)

wp.launch(
    iterate_vertex_neighbor_primitives,
    dim=model.particle_count,
    inputs=[vbd_integrator.adjacency],
    device=device
)

Module __main__ 3dd20ff load on device 'cuda:0' took 22.56 ms  (cached)


In [57]:
# your turn: you need to:
# Implement conservative bounds computation using the instructions provided above
# it must be implemented using @warp.kernel to maximize efficiency

#### Computing Conservative Bounds

In [ ]:
import warp as wp
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo

@wp.kernel
def compute_conservative_bounds(
    # Current and previous vertex positions
    x_current: wp.array(dtype=wp.vec3),
    x_prev: wp.array(dtype=wp.vec3),
    
    # Pre-computed minimum distances from collision detector
    d_min_v: wp.array(dtype=wp.float32),  # vertex to non-adjacent triangles
    d_min_e: wp.array(dtype=wp.float32),  # edge to non-adjacent edges  
    d_min_t: wp.array(dtype=wp.float32),  # triangle to non-adjacent vertices
    
    # Adjacency information
    adjacency: ForceElementAdjacencyInfo,
    
    # Parameters
    gamma_p: wp.float32,  # relaxation parameter (0 < gamma_p < 0.5)
    
    # Output: conservative bounds for each vertex
    bounds: wp.array(dtype=wp.float32)
):
    vertex_idx = wp.tid()
    
    # Get current and previous positions for this vertex
    x_v = x_current[vertex_idx]
    x_prev_v = x_prev[vertex_idx]
    
    # Initialize minimum distances for this vertex's neighbors
    d_min_e_neighbor = wp.float32(1e10)  # Large initial value
    d_min_t_neighbor = wp.float32(1e10)  # Large initial value
    
    # Get number of adjacent edges for this vertex
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, vertex_idx)
    
    # Iterate over neighbor edges to find minimum d_min_e among neighbors
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, vertex_idx, edge_counter)
        
        # Get the minimum distance for this neighboring edge
        edge_d_min = d_min_e[edge_idx]
        if edge_d_min < d_min_e_neighbor:
            d_min_e_neighbor = edge_d_min
    
    # Get number of adjacent faces for this vertex  
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, vertex_idx)
    
    # Iterate over neighbor faces to find minimum d_min_t among neighbors
    for face_counter in range(num_adj_faces):
        face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, vertex_idx, face_counter)
        
        # Get the minimum distance for this neighboring face
        face_d_min = d_min_t[face_idx]
        if face_d_min < d_min_t_neighbor:
            d_min_t_neighbor = face_d_min
    
    # Compute conservative bound according to equation (21)
    # b_v = gamma_p * min(d_min_v, d_min_e_neighbor, d_min_t_neighbor)
    
    min_distance = d_min_v[vertex_idx]
    if d_min_e_neighbor < min_distance:
        min_distance = d_min_e_neighbor
    if d_min_t_neighbor < min_distance:
        min_distance = d_min_t_neighbor
    
    # Apply relaxation parameter
    conservative_bound = gamma_p * min_distance
    
    # Store the result
    bounds[vertex_idx] = conservative_bound


def compute_bounds_for_simulation(model, collision_detector, vbd_integrator, gamma_p=0.3):
    """
    Compute conservative bounds for all vertices in the simulation
    
    Args:
        model: Warp simulation model
        collision_detector: TriMeshCollisionDetector instance with computed distances
        vbd_integrator: VBD integrator containing adjacency information
        gamma_p: Relaxation parameter (0 < gamma_p < 0.5)
    
    Returns:
        Array of conservative bounds for each vertex
    """
    device = wp.get_device('cuda')
    
    # Get current simulation state
    state = model.state()
    
    # Allocate output array for bounds
    bounds = wp.zeros(model.particle_count, dtype=wp.float32, device=device)
    
    # Launch the kernel
    # Note: Using current positions for both current and previous for initial computation
    # In a real simulation, you would store the previous positions from the last time step
    wp.launch(
        compute_conservative_bounds,
        dim=model.particle_count,
        inputs=[
            state.particle_q,      # current positions
            state.particle_q,      # using current positions as previous (for initial step)
            collision_detector.vertex_colliding_triangles_min_dist,  # d_min_v
            collision_detector.edge_colliding_edges_min_dist,       # d_min_e  
            collision_detector.triangle_colliding_vertices_min_dist, # d_min_t
            vbd_integrator.adjacency,
            gamma_p,
            bounds
        ],
        device=device
    )
    
    return bounds


In [16]:
# After your collision detection setup
bounds = compute_bounds_for_simulation(
    model, 
    colision_detector, 
    vbd_integrator,
    gamma_p=0.3
)

In [ ]:
import polyscope as ps
ps.init()
mesh_ps = ps.register_surface_mesh("mesh", mesh.vertices, mesh.faces)
mesh_ps.add_scalar_quantity("Bounds", bounds.numpy(),cmap='viridis')
ps.show()